In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from pandas_profiling import ProfileReport
import tensorflow as tf
from sklearn.feature_extraction import text
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import re
from wordcloud import WordCloud, STOPWORDS
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import sklearn.decomposition as decomposition
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/byop-data/Online_Retail.csv


In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver() 
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy() 

print("REPLICAS: ", strategy.num_replicas_in_sync)

In [3]:
data=pd.read_csv('/kaggle/input/byop-data/Online_Retail.csv', encoding="windows-1252")
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,01-12-2010 08:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,01-12-2010 08:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,01-12-2010 08:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,01-12-2010 08:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,01-12-2010 08:26,3.39,17850.0,United Kingdom


In [ ]:
# Data Description
data.describe()
# notice negative values in Quantity and UnitPrice

In [ ]:
#Doing value count for various variables
data['Country'].value_counts()

In [ ]:
data['StockCode'].value_counts()

In [ ]:
#Data Profiling
profile = ProfileReport(data, title= "Online Retail Profile Report")
profile

In [ ]:
profile.to_file("BYOP_Profile.html")

In [ ]:
# Percentage of Missing Values
data.isnull().sum()/len(data)

In [ ]:
#Exploring Negative Values
data.where(data['UnitPrice']<0).describe() #only 2 counts of negative UnitPrice for which Quantity is 1

In [ ]:
data.where(data['Quantity']<0).describe()
# 10624 entries in Qunatity have negative values, out of which 8905 have CustomerID
#?? Why would a customer be invoiced a negative quntity? Are these refunds/returns?
#?? Then these customer IDs must match a corresponding invoice when items were bought? - Matching CustomerID and Quantity found for some
#but not all

In [4]:
# Create new column with total revenue per customer
data['Revenue*'] = data['Quantity']*data['UnitPrice']
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Revenue*
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,01-12-2010 08:26,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,01-12-2010 08:26,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,01-12-2010 08:26,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,01-12-2010 08:26,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,01-12-2010 08:26,3.39,17850.0,United Kingdom,20.34


In [4]:
# Arranging data by CustomerID for highest spends (after accounting for returns)
grouped_data=data.groupby(by= 'CustomerID', sort=True)
data_rev_perID=grouped_data.sum()
data_rev_perID.sort_values(by='Revenue*', ascending=False)
# ??Output  has only 4372 rows, original data has 4373 rows
#en(data['CustomerID'].unique())

,Quantity,UnitPrice,Revenue*
CustomerID,,,
14646.0,196719,5400.21,279489.02
18102.0,64122,5159.73,256438.49
17450.0,69029,3320.09,187482.17
14911.0,77180,31060.66,132572.62
12415.0,77242,2499.82,123725.45
...,...,...,...
12503.0,-1,1126.00,-1126.00
17603.0,-31,1156.89,-1165.30
14213.0,-244,24.45,-1192.20


In [5]:
#NLP Part

#get text data from Description column and convert to string dtype and lowercase
text_data=data['Description'].astype("str")
text_data = text_data.apply(lambda x: x.lower())
text_data.head()

0     white hanging heart t-light holder
1                    white metal lantern
2         cream cupid hearts coat hanger
3    knitted union flag hot water bottle
4         red woolly hottie white heart.
Name: Description, dtype: object

In [6]:
# function to remove special characters
text_data= text_data.apply(lambda x: re.sub("[^0-9a-zA-Z]",' ', x))

In [7]:
# removing stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words("english"))
stop_words

def remove_sw(review):
    tokens = word_tokenize(review)
    tokens = [w for w in tokens if not w in stop_words]
    return " ".join(tokens)
text_data = text_data.apply(remove_sw)

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
# Generate word cloud
wc = " ".join(cat for cat in text_data)
print(type(wc))

<class 'str'>


In [10]:
word_cloud = WordCloud(width=3000,height=2000,random_state=1,background_color="salmon",colormap="Pastel1",collocations=True,
        stopwords=STOPWORDS,max_words=50
        ).generate(wc)

In [11]:
plt.imshow(word_cloud, interpolation='bilinear')
plt.axis("off")
plt.show()
%matplotlib inline

In [13]:
#TF-IDF/Count vector only work on a list
text_data=text_data.tolist()
#checking for Nan values
#text_data.count(np.nan)

AttributeError: 'list' object has no attribute 'tolist'

In [14]:
# Initialise CountVectorizer
cv= text.CountVectorizer(input=text_data, ngram_range=(1,2), stop_words='english')
matrix_cv = cv.fit_transform(text_data)
matrix_cv

<541909x9282 sparse matrix of type '<class 'numpy.int64'>'
	with 3835735 stored elements in Compressed Sparse Row format>

In [ ]:
#checking if count vector matrix has captured all rows
#cv_check=pd.DataFrame(matrix_cv.toarray(), columns=cv.get_feature_names())
#cv_check.shape

In [15]:
# initialise TF-IDF
tf=text.TfidfVectorizer(input=text_data,ngram_range=(4,4),stop_words='english')
matrix_tfidf=tf.fit_transform(text_data)

In [ ]:
matrix_tfidf.shape

In [ ]:
Sum_of_squared_distances = []
K = range(30,50)
for num_clusters in K :
 model_check = KMeans(n_clusters=num_clusters)
 model_check.fit(matrix_tfidf)
 Sum_of_squared_distances.append(model_check.inertia_)

In [ ]:
plt.plot(K,Sum_of_squared_distances)
plt.xlabel('Values of K') 
plt.ylabel('Sum of squared distances/Inertia') 
plt.title('Elbow Method For Optimal k')
plt.show()

In [16]:
# Initiating K Means Clustering
kmeans = KMeans(n_clusters=32, random_state=42)
# fit the model
kmeans.fit(matrix_tfidf)
# store cluster labels in a variable
clusters = kmeans.labels_

In [18]:
data['cluster']=clusters

In [19]:
# fetching relevant keywords for each cluster

def get_top_keywords(n_terms):
    """This function returns the keywords for each centroid of the KMeans"""
    df = pd.DataFrame(matrix_tfidf.todense()).groupby(clusters).mean() # groups the TF-IDF vector by cluster
    terms = tf.get_feature_names_out() # access tf-idf terms
    for i,r in df.iterrows():
        print('\nCluster {}'.format(i))
        print(','.join([terms[t] for t in np.argsort(r)[-n_terms:]])) # for each row of the dataframe, find the n terms that have the highest tf idf score
            
get_top_keywords(8)


Cluster 0
roses regency teacup saucer,jam making set printed,french blue metal door,blue metal door sign,lunch bag spaceboy design,jumbo storage bag suki,lunch bag suki design,lunch bag red retrospot

Cluster 1
felt hanging heart flower,felt flower peg bag,felt farm animal white,felt farm animal sheep,felt farm animal rabbit,zinc wire sweetheart letter,cake tins pantry design,set cake tins pantry

Cluster 2
feltcraft girl nicole kit,feltcraft hairband pink blue,feltcraft hairband pink purple,feltcraft hairband red blue,felt farm animal hen,zinc wire sweetheart letter,kneeling pad cup tea,gardeners kneeling pad cup

Cluster 3
felt hanging multicol garland,felt hanging heart flower,felt flower peg bag,felt farm animal white,felt farm animal sheep,feltcraft boy jean paul,zinc wire sweetheart letter,jumbo bag red retrospot

Cluster 4
feltcraft hairband pink blue,feltcraft hairband pink purple,feltcraft hairband red blue,feltcraft hairbands pink white,felt farm animal rabbit,zinc wire swee

In [ ]:
#checking if tf-idf matrix has captured all rows
#tf_check=pd.DataFrame(matrix_tfidf.toarray(), columns=tf.get_feature_names())
#tf_check.shape

**# RFM Analysis**

In [ ]:
# Creating a copy of the data
rfm=data

In [ ]:
rfm.info()

In [ ]:
#Dropping missing values
rfm=rfm.dropna()
rfm.info()

In [ ]:
rfm.head()

In [ ]:
# Converting InvoiceDate to date-time object
from datetime import datetime
rfm['InvoiceDate']=rfm['InvoiceDate'].apply(lambda x: x.split(' ')[0])
rfm['InvoiceDate'].head()

In [ ]:
# rfm['InvoiceDate']=rfm['InvoiceDate'].apply(lambda x:datetime.strptime(x, '%d-%m-%y'))
# ValueError: unconverted data remains: 10

In [ ]:
from dateutil import parser

rfm['InvoiceDate']=rfm['InvoiceDate'].apply(lambda x:parser.parse(x))

In [ ]:
rfm.info()

In [ ]:
# Aggregate at customer/customerID level
customer_data= pd.DataFrame(rfm.pivot_table(index='CustomerID',values=['InvoiceDate', 'Revenue*'], 
                                            aggfunc={'InvoiceDate':[min, max, pd.Series.nunique], 'Revenue*':sum}))
customer_data.head()

In [ ]:
customer_data.info()

In [ ]:
# Rename columns
customer_data.columns = ["LastInvoiceDate", "FirstInvoiceDate", "Frequency", "MonetaryValue"]
customer_data.head()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
# Create column "FirstInvoceMonth" to look at customer intake
customer_data["FirstInvoiceMonth"] = customer_data["FirstInvoiceDate"].apply(lambda x: x.replace(day=1))
customer_data.groupby(["FirstInvoiceMonth"]).count()["FirstInvoiceDate"].plot(kind="bar")
plt.title("Monthly Customer Intakes")

In [ ]:
# Calculate recency, relative recency and relative frequency
# Take the maximum invoice date as today
today = customer_data["LastInvoiceDate"].max()
today

In [ ]:
# Calculate recency: the interval (days) between the last transation day and today
customer_data["Recency"] = (today - customer_data["LastInvoiceDate"]) / np.timedelta64(1, 'D')
customer_data["Recency"].hist()

In [ ]:
# Calculate customer lifetime: the interval (days) between the first transation day and today
customer_data["Lifetime"] = (today - customer_data["FirstInvoiceDate"]) / np.timedelta64(1, 'D')
customer_data["Lifetime"].hist()

In [ ]:
customer_data["Lifetime"].describe()

In [ ]:
# Calculate the relative recency: recency / customer lifetime
# Note: this is to normalise the fact that newer customers have lower lifetime and thus lower recency
# by nature.
customer_data["RelRecency"] = 1 - customer_data["Recency"] / customer_data["Lifetime"]
customer_data["RelRecency"].hist()

In [ ]:
customer_data["Recency"].describe()

In [ ]:
customer_data["RelRecency"].describe()
# Why count goes from 4372 to 4367?

In [ ]:
sum(customer_data["RelRecency"]==0) / len(customer_data)
# This shows that 31% customers only had one transactions.

In [ ]:
# Calculate the relative frequency: frequency / customer lifetime
# Note: this is to normalise the fact that newer customers have lower lifetime and thus lower frequency
# by nature.
customer_data["RelFrequency"] = customer_data["Frequency"] / customer_data["Lifetime"]
customer_data["RelFrequency"].apply(lambda x: np.isinf(x)).value_counts()

In [ ]:
# Found 5 records with inifinity value, we need to remove these
customer_data[customer_data["RelFrequency"].apply(lambda x: np.isinf(x))]
# Meaning these customers got acquired today only?

In [ ]:
customer_data = customer_data[customer_data["RelFrequency"].apply(lambda x: np.isinf(x))==False]
customer_data["RelFrequency"].hist()

In [ ]:
customer_data["RelFrequency"].describe()

In [ ]:
# Calculate the relative monetary value: monetary value / customer lifetime
# Note: this is to normalise the fact that newer customers have lower lifetime and thus lower frequency
# by nature.
customer_data["MonetaryValue"].describe()

In [ ]:
customer_data["MonetaryValue"].hist()
# There are some outliers in terms of monetary value.

In [ ]:
np.percentile(customer_data["MonetaryValue"], 99.5)
# The 99.5% percentile of monetary value is 33.4K, i.e., 0.5% * 4K = 20 customers have value higher than 33.4K.
# We should exclude these outliers from our analysis.

In [ ]:
customer_data = customer_data[customer_data["MonetaryValue"]<=np.percentile(customer_data["MonetaryValue"], 99)]

In [ ]:
len(customer_data)

In [ ]:
len(customer_data[customer_data["MonetaryValue"]<0])
# 42 customers have negative transation value because of the returns.

In [ ]:
customer_data[customer_data["MonetaryValue"]<0].head()

In [ ]:
data[data["CustomerID"]==12605]
# Some customers have a negative sum of order value. This is because they have returns.

In [ ]:
customer_data["RelMonetaryValue"] = customer_data["MonetaryValue"] / customer_data["Lifetime"]
customer_data["RelMonetaryValue"].hist()

In [ ]:
customer_data["RelMonetaryValue"].describe()

In [ ]:
# Create RFM buckets using absolute values
# For this analysis, we take the medians and 75% quartiles of relative recency, relative frequency and relative monetary value and use them for
# 4 bins for each variable. We label those bins as 1, 2, 3 and 4 and use them as the scores for R, F and M respectively.
# We then create 
# By doing so we end up with 10 clusters ()

In [ ]:
customer_data['RecencyScore'] = pd.cut(customer_data["Recency"], 
                                       bins=[-1, 
                                             np.percentile(customer_data["Recency"], 25), 
                                             np.percentile(customer_data["Recency"], 50), 
                                             np.percentile(customer_data["Recency"], 75), 
                                             customer_data["Recency"].max()], 
                                       labels=[4, 3, 2, 1]).astype("int")
customer_data["RecencyScore"].value_counts()

In [ ]:
customer_data["FrequencyScore"] = pd.cut(customer_data["Frequency"], 
                                       bins=[-1, 
                                             np.percentile(customer_data["Frequency"], 25), 
                                             np.percentile(customer_data["Frequency"], 50), 
                                             np.percentile(customer_data["Frequency"], 75), 
                                             customer_data["Frequency"].max()], 
                                       labels=[1, 2, 3, 4]).astype("int")
customer_data["FrequencyScore"].value_counts()

In [ ]:
customer_data["MonetaryScore"] = pd.cut(customer_data["MonetaryValue"], 
                                       bins=[customer_data["MonetaryValue"].min()-1, 
                                             np.percentile(customer_data["MonetaryValue"], 25),
                                             np.percentile(customer_data["MonetaryValue"], 50), 
                                             np.percentile(customer_data["MonetaryValue"], 75), 
                                             customer_data["MonetaryValue"].max()], 
                                       labels=[1, 2, 3, 4]).astype("int")
customer_data["MonetaryScore"].value_counts()

In [ ]:
customer_data["RFM"] = customer_data["RecencyScore"] + customer_data["FrequencyScore"] + customer_data["MonetaryScore"]
customer_data["RFM"].value_counts()

In [ ]:
rfm_abs = pd.DataFrame(customer_data.pivot_table(index=["RFM"], 
                                    values=["Recency", "Frequency", "MonetaryValue", "Lifetime"], 
                                    aggfunc={"Recency": [np.min, np.median, np.max], 
                                             "Frequency": [np.min, np.median, np.max], 
                                             "MonetaryValue": [np.min, np.median, np.max], 
                                             "Lifetime": [np.min, np.median, np.max, "count"]}))
rfm_abs

In [ ]:
#As we can see, the groups have very different median days of lifetime. This suggests potential bias associated with customer sign up date.

In [ ]:
# Create RFM buckets using relative values
# For this analysis, we take the medians and 75% quartiles of relative recency, relative frequency and relative monetary value and use them for
# 4 bins for each variable. We label those bins as 1, 2, 3 and 4 and use them as the scores for R, F and M respectively.
# We then create 
# By doing so we end up with 10 clusters ()
customer_data["RecencyScore"] = pd.cut(customer_data["RelRecency"], 
                                       bins=[-1, 
                                             np.percentile(customer_data["RelRecency"], 25), 
                                             np.percentile(customer_data["RelRecency"], 50), 
                                             np.percentile(customer_data["RelRecency"], 75), 
                                             customer_data["RelRecency"].max()], 
                                       labels=[1, 2, 3, 4]).astype("int")
customer_data["RecencyScore"].value_counts()

In [ ]:
customer_data["FrequencyScore"] = pd.cut(customer_data["RelFrequency"], 
                                       bins=[-1, 
                                             np.percentile(customer_data["RelFrequency"], 25), 
                                             np.percentile(customer_data["RelFrequency"], 50), 
                                             np.percentile(customer_data["RelFrequency"], 75), 
                                             customer_data["RelFrequency"].max()], 
                                       labels=[1, 2, 3, 4]).astype("int")
customer_data["FrequencyScore"].value_counts()

In [ ]:
customer_data["MonetaryScore"] = pd.cut(customer_data["RelMonetaryValue"], 
                                       bins=[customer_data["RelMonetaryValue"].min()-1, 
                                             np.percentile(customer_data["RelMonetaryValue"], 25),
                                             np.percentile(customer_data["RelMonetaryValue"], 50), 
                                             np.percentile(customer_data["RelMonetaryValue"], 75), 
                                             customer_data["RelMonetaryValue"].max()], 
                                       labels=[1, 2, 3, 4]).astype("int")
customer_data["MonetaryScore"].value_counts()

In [ ]:
customer_data.head()

In [ ]:
customer_data["RFM"] = customer_data["RecencyScore"] + customer_data["FrequencyScore"] + customer_data["MonetaryScore"]
customer_data["RFM"].value_counts()

In [ ]:
rfm_rel = pd.DataFrame(customer_data.pivot_table(index=["RFM"], 
                                    values=["Recency", "Frequency", "MonetaryValue", "Lifetime"], 
                                    aggfunc={"Recency": [np.min, np.median, np.max], 
                                             "Frequency": [np.min, np.median, np.max], 
                                             "MonetaryValue": [np.min, np.median, np.max], 
                                             "Lifetime": [np.min, np.median, np.max, "count"]}))
rfm_rel
# Note that the median lifetime is rather constant across clusters. This is a good news - our segmentation is not biased by the lifetime
# of the customers.

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=1)
fig.suptitle('Median Lifetime of RFM Segments (Absolute vs Relative)')
rfm_abs["Lifetime"]["median"].plot(ax=axes[0], kind="bar")
rfm_rel["Lifetime"]["median"].plot(ax=axes[1], kind="bar")
plt.show()

In [ ]:
# Visualise segments using 3D plot
x = customer_data["RelRecency"]
y = customer_data["RelFrequency"]
z = (customer_data["RelMonetaryValue"] - customer_data["RelMonetaryValue"].min()) / customer_data["RelMonetaryValue"].max()
c = customer_data["RFM"]
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(xs=x, ys=y, zs=z, c=c, s=30, alpha=0.5)
ax.set_title("RFM Visualisation")
ax.set_xlabel("Relative Recency")
ax.set_ylabel("Relative Frequency")
ax.set_zlabel("Relative Monetary Value (with Min-Max Standardisation)")
plt.show()


In [ ]:
fig, axs = plt.subplots(nrows=3, ncols=1, sharex=True, sharey=True, figsize=(5, 15))
fig.suptitle('Correlation between R, F and M')

axs[0].scatter(x, y, alpha=0.5)
axs[0].set_title("Relative Recency (x) vs Relative Frequency (y)")

axs[1].scatter(y, z, alpha=0.5)
axs[1].set_title("Relative Frequency (x) vs Relative Monetary Value (y)")

axs[2].scatter(x, z, alpha=0.5)
axs[2].set_title("Relative Recency (x) vs Relative Monetary Value (y)")

In [ ]:
x = customer_data["RecencyScore"]
y = customer_data["FrequencyScore"]
z = customer_data["MonetaryScore"]
c = customer_data["RFM"]
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(xs=x, ys=y, zs=z, c=c, s=30, alpha=0.5)
ax.set_title("RFM Visualisation")
ax.set_xlabel("Recency Score")
ax.set_ylabel("Frequency Score")
ax.set_zlabel("Monetary Score")
plt.show()

In [ ]:
# From these statistics we can generate some heuristic business rules to simplify our segmentation.
# For example, RFM score 3 and 4 can be combined, because these two groups tend to have only one transaction.
# Create new columns with our learnings
customer_data.describe()

In [ ]:
rfm_rel

In [ ]:
customer_data["last_order_within_l60d"] = customer_data["Recency"]<60 # Had transactions in the last 60 days
customer_data["more_than_two_orders"] = customer_data["Frequency"]>2 # Logged in more than twice
customer_data["value_higher_than_2k"] = customer_data["MonetaryValue"]>2000 # Sum of value higher than 2K
customer_data.groupby(["last_order_within_l60d", "more_than_two_orders", "value_higher_than_2k"]).count()["Lifetime"]

In [ ]:
conditions = [
    (customer_data["last_order_within_l60d"]==True)&(customer_data["more_than_two_orders"]==True)&(customer_data["value_higher_than_2k"]==True),
    (customer_data["last_order_within_l60d"]==True)&(customer_data["more_than_two_orders"]==True)&(customer_data["value_higher_than_2k"]==False),
    (customer_data["last_order_within_l60d"]==True)&(customer_data["more_than_two_orders"]==False),
    (customer_data["last_order_within_l60d"]==False)&(customer_data["more_than_two_orders"]==True),
    (customer_data["last_order_within_l60d"]==False)&(customer_data["more_than_two_orders"]==False)
]
mappings = ["01. high engagement & high value", 
            "02. high engagement & low value", 
            "03. recent and low frequency", 
            "04. old and high frequency", 
            "05. low engagement & low value"]
customer_data['FinalRFM'] = np.select(conditions, mappings, default="Others")
customer_data['FinalRFM'].value_counts()

In [ ]:
customer_data.pivot_table(index=["FinalRFM"], 
                          values=["Recency", "Frequency", "MonetaryValue", "Lifetime"], 
                          aggfunc={"Recency": [np.min, np.median, np.max], 
                                   "Frequency": [np.min, np.median, np.max], 
                                   "MonetaryValue": [np.min, np.sum,np.median, np.max], 
                                   "Lifetime": [np.min, np.median, np.max, "count"]})

In [ ]:
# Calculate the number of new users, inactive users, return users and active users in each month
# Definition:
# New users: those who made their first purchase in the current month
# Active users: those who made purchases in the previous month and in the current month
# Inactive users: those who made purchases in previous months, but not in the current month
# Return users: those who made purchases before the previous month, not in the previous month and made purchases agian in the current month
#data["InvoiceMonth"] = data["InvoiceDate"].apply(lambda x: x.replace(day=1))
#data["InvoiceMonth"] = data["InvoiceMonth"].apply(lambda x: x.strftime("%Y-%m-%d"))
#user_month_pivot = data.pivot_table(index=["CustomerID"], 
#                                    columns=["InvoiceMonth"], 
#                                    values=["InvoiceNo"], 
#                                    aggfunc="count", 
#                                    fill_value=0)
#user_month_pivot.head()

In [ ]:
rfm.head()